# Empowerment: Pacman

In [3]:
import numpy as np
from itertools import combinations_with_replacement, combinations, permutations, product

In [4]:
# ' ' - nothing
# 'P' - player
# '#' - wall
# 'G' - ghost

field = np.array(
        [[' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', '#', '#', '#', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#', ' '],
         [' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#', ' '],
         [' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#', ' '],
         [' ', ' ', '#', ' ', '#', '#', '#', '#', ' ', ' ', ' ', '#', 'G', '#', ' '],
         ['#', '#', '#', 'P', '#', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#', ' '],
         [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' '],
         [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', '#', '#', '#', '#', ' ', ' '],
         [' ', ' ', ' ', ' ', '#', ' ', ' ', 'G', ' ', ' ', ' ', ' ', '#', ' ', ' '],
         [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', '#', ' '],
         [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' '],
         [' ', ' ', '#', '#', '#', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' '],
         [' ', ' ', '#', ' ', ' ', ' ', '#', '#', '#', '#', ' ', ' ', ' ', '#', ' '],
         [' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'G', ' ']])

height = len(field)
width = len(field[0])

In [5]:
steps = 4

def update(current):
    return current

def actions_P(field):
    # position of variable (index)
    Px, Py = np.where(field == 'P')
    # field[int(x)][int(y)] 
    
    # number of possible actions: 4
    #    -0-
    #    3P1
    #    -2-
    # calculation of possible steps from future position (0 to 3)
    paths = set()
    for i in product(range(steps), repeat=steps):
        paths.add(i)
    
    # iterate over field around P
    # 1. calculate free postions around you, 2. calculate number of actions possible from free positions
    counts = []

    
    # for i in range(height):
        # for j in range(width):
            
    return

In [6]:
paths = set()
for i in product(range(4), repeat=4):
    paths.add(i)

In [11]:
# Returns the resulting position of a moving object, given its current position and its action
def update_position(position, action):
    newpos = [position[0] + action[0], position[1] + action[1]]
    newpos[0] %= height
    newpos[1] %= width
    return newpos

In [39]:
def player_position(field):
    return np.array(np.where(field == 'P')).flatten()

In [41]:
# Returns a list of possible actions (lists with relative y and x positions), given the current field and angent position
def possible_actions(field, position):
    actions = []
    for action in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
        newpos = update_position(position, action)
        target = field[newpos[0]][newpos[1]]
        if target == '#':
            continue
        elif target == 'G':
            continue
        actions.append(action)
    return actions

In [48]:
def update(field, action):
    return field

In [46]:
# Returns an array of fields which result in one of the possible player actions being taken
# (starting from every field in the input array)
def possible_fields(fields):
    possibilities = []
    for field in fields:
        for action in possible_actions(field, player_position(field)):
            possibilities.append(update(field, action))
    return possibilities

In [72]:
# Returns an empowerment measure calculated from an array of reachable fields
def empowerment(fields):
    return np.log(len(set([tuple(tuple(element) for element in field) for field in fields])))
    
#print(empowerment(possible_fields([field, field])))